In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import AlignmentCalculator
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7526924411681656086"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [3]:
video_path = os.path.abspath(f"./tmp/video/{video_id}.mp4")
audio_path = os.path.abspath(f"./tmp/video/{video_id}.wav")

transcript = """
This is where we hold them, this is where we fight, this is where they die, as these shields fall.
"""

sentiment_results = {
    'positive': 0.01686652,
    'neutral': 0.2764731,
    'negative':0.70666033
 }

description = '#doamneajuta #🙏❤️🇷🇴 #@calin.georgescu.real #suveranitate #romania #fyp #OnoaresiRespect🙏❤️🇷🇴 '

In [4]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)
analysis_result_json

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7526924411681656086.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': 'The video alleges that on July 14, 2025, Romania was not honest, and on July 20, 2025, Bogdan Peschir was arrested for donations in Bucharest. It further claims that Stegarul Dac was arrested for waving the Romanian flag and that Călin Georgescu was annulled by the CCR and blocked by BEC. The video questions whether this is the Romania people want, asking how much more can be accepted and why Romanians cannot stand up for themselves, concluding w

{'summary': 'The video alleges that on July 14, 2025, Romania was not honest, and on July 20, 2025, Bogdan Peschir was arrested for donations in Bucharest. It further claims that Stegarul Dac was arrested for waving the Romanian flag and that Călin Georgescu was annulled by the CCR and blocked by BEC. The video questions whether this is the Romania people want, asking how much more can be accepted and why Romanians cannot stand up for themselves, concluding with a call for "DEMISIA" (Resignation).',
 'identified_subjects': [{'subject': 'calin georgescu', 'stance': -1.0},
  {'subject': 'romania', 'stance': -1.0},
  {'subject': 'national sovereignty', 'stance': -1.0},
  {'subject': 'ccr', 'stance': -1.0},
  {'subject': 'bogdan peschir', 'stance': -1.0}],
 'overall_alignment': -1.0}

In [5]:
identified_subjects = analysis_result_json["identified_subjects"]
llm_overall_alignment = analysis_result_json["overall_alignment"]
final_alignment, deterministic_alignment, alignment_conflict = AlignmentCalculator.calculate(identified_subjects, llm_overall_alignment, alpha=0.25)
print(f"llm_overall_alignment: {llm_overall_alignment} \nfinal_alignment: {final_alignment} \ndeterministic_alignment: {deterministic_alignment} \nalignment_conflict: {alignment_conflict}")

llm_overall_alignment: -1.0 
final_alignment: 0.9879518072289156 
deterministic_alignment: 0.9879518072289156 
alignment_conflict: 1.9879518072289155
